<a href="https://colab.research.google.com/github/OMONa-E/Prediction-of-Product-Sales/blob/main/Omona_Emmanuel_Prediction_of_Product_Sales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prediction of Product Sales

- Author: Omona Emmanuel

## Project Overview


## Load and Inspect Data


## Clean Data


## Exploratory Data Analysis


## Feature Inspection